MÁSTER BIG DATA & DATA ENGINEERING

Entrega Final Módulo: Programación avanzada en Python

Autor: Germán Artigot Cortizo



## Nota Preliminar

Según se observaré en la evaluación de este proyecto, a lo largo del mismo se recurren a distintos métodos para realizar funciones similares. Esto no es sino debido a que el proyecto se ha realizado a lo largo de varios días por lo que, según se profundizaba en el conocimiento y la práctica de la programación con Pandas, se han hallado diversas formas de lograr el mismo resultado.
